# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

In [38]:
%%writefile ../flows/cloud/taxi_champion.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger
from metaflow import project, S3
from metaflow.cards import Markdown, Table, Image, Artifact

# URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
URL = 's3://outerbounds-datasets/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

@trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2'})
@project(name='taxifare_prediction')
class TaxiFarePrediction(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):
       

        obviously_bad_data_filters = {
            'fare_amount': df.fare_amount > 0,         # fare_amount in US Dollars
            'trip_distance_max': df.trip_distance <= 100,    # trip_distance in miles
            'trip_distance_min': df.trip_distance > 0,
            'tip_amount': df.tip_amount >= 0,
            'total_amount': df.total_amount > 0,
            'tolls_amount': df.tolls_amount >= 0,
        }

        for key, f in obviously_bad_data_filters.items():
            df = df[f]
            # print(f'Removed {key}, size: {len(df)}')

        return df

    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split

        with S3() as s3:
            obj = s3.get(URL)
            df = pd.read_parquet(obj.path)

        self.df = self.transform_features(df)
        # self.df = self.transform_features(pd.read_parquet(self.data_url))

        # NOTEOK: we are split into training and validation set in the validation step which uses cross_val_score.
        
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import LinearRegression

        # TODODONE: Play around with the model if you are feeling it.
        self.model = LinearRegression()
        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        self.model_name = 'champion'
        print(f'Score: {self.scores.mean():.4f}')
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ../flows/cloud/taxi_champion.py


In [13]:
! python ../flows/cloud/taxi_champion.py --environment=conda run


Metaflow 2.9.2+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxifare_prediction, Branch: user.sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Bootstrapping conda environment...(this could take a few minutes)
2023-05-24 01:47:12.259 Workflow starting (run-id 34), see it in the UI at https://ui-pw-555244755.outerbounds.dev/TaxiFarePrediction/34
2023-05-24 01:47:13.206 [34/start/141 (pid 24313)] Task is starting.
2023-05-24 01:47:24.336 [34/start/141 (pid 24313)] Task finished successfully.
2023-05-24 01:47:25.263 [34/linear_model/142 (pid 24422)] Task is starting.
2023-05-24 01:47:27.734 [34/linear_model/142 (pid 24422)] Task finished successfully.
2023-05-24 01:47:28.594 [34/validate/143 (pid 24466)] Task is starting.
2023-05-24 01:47:34.364 [34/validate/143 (pid 24466)] Task finished successfully.
2023-05-24 01:47:35.222 [34/end/144 (pid 24530)] Task is starting.
2023-05-24 01:47:36.430 [34/end/144 (pid 24530)] Score: 0.

In [15]:
! python ../flows/cloud/taxi_champion.py --environment=conda --production --branch champion --production argo-workflows create


Metaflow 2.9.2+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxifare_prediction, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taxifareprediction.prod.champion.taxifareprediction to Argo Workflows...

The namespace of this production flow is
    production:mfprj-bl5jeer7xehnsiac-0-hqum
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-bl5jeer7xehnsiac-0-hqum")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-bl5jeer7xehnsiac-0-hqum
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping conda environment...(this could take a few minutes)
Workflow taxifareprediction.prod.champion.taxifareprediction for flow TaxiFar

In [16]:
! python ../flows/cloud/taxi_champion.py --environment=conda --production --branch champion argo-workflows trigger


Metaflow 2.9.2+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxifare_prediction, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taxifareprediction.prod.champion.taxifareprediction triggered on Argo Workflows (run-id argo-taxifareprediction.prod.champion.taxifareprediction-ljptv).
See the run in the UI at https://ui-pw-555244755.outerbounds.dev/TaxiFarePrediction/argo-taxifareprediction.prod.champion.taxifareprediction-ljptv


## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

In [39]:
%%writefile ../flows/cloud/taxi_challenger.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger
from metaflow import project, S3
from metaflow.cards import Markdown, Table, Image, Artifact

# URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
URL = 's3://outerbounds-datasets/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

@trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2', 'xgboost' : '1.7.4'})
@project(name='taxifare_prediction')
class TaxiFarePrediction(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):
        # TODODONE: 
        # Try to complete tasks 2 and 3 with this function doing nothing like it currently is.
        # Understand what is happening.
        # Revisit task 1 and think about what might go in this function.

        obviously_bad_data_filters = {
            'fare_amount': df.fare_amount > 0,         # fare_amount in US Dollars
            'trip_distance_max': df.trip_distance <= 100,    # trip_distance in miles
            'trip_distance_min': df.trip_distance > 0,
            'tip_amount': df.tip_amount >= 0,
            'total_amount': df.total_amount > 0,
            'tolls_amount': df.tolls_amount >= 0,
        }

        for key, f in obviously_bad_data_filters.items():
            df = df[f]
            # print(f'Removed {key}, size: {len(df)}')

        return df

    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split

        with S3() as s3:
            obj = s3.get(URL)
            df = pd.read_parquet(obj.path)

        self.df = self.transform_features(df)
        # self.df = self.transform_features(pd.read_parquet(self.data_url))

        # NOTEOK: we are split into training and validation set in the validation step which uses cross_val_score.
      
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.xgboost_model)
    
    @step
    def xgboost_model(self):
        "Fit an XGBoost regression"
        from xgboost import XGBRegressor
        self.model = XGBRegressor()
        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score

        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        self.model_name = 'challenger'
        print(f'{self.model_name} score: {self.scores.mean():.4f}')
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ../flows/cloud/taxi_challenger.py


In [31]:
! python ../flows/cloud/taxi_challenger.py --environment=conda run



Metaflow 2.9.2+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxifare_prediction, Branch: user.sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Bootstrapping conda environment...(this could take a few minutes)
2023-05-24 02:01:32.811 Workflow starting (run-id 38), see it in the UI at https://ui-pw-555244755.outerbounds.dev/TaxiFarePrediction/38
2023-05-24 02:01:33.734 [38/start/160 (pid 25508)] Task is starting.
2023-05-24 02:01:40.572 [38/start/160 (pid 25508)] Task finished successfully.
2023-05-24 02:01:41.482 [38/xgboost_model/161 (pid 25598)] Task is starting.
2023-05-24 02:01:44.046 [38/xgboost_model/161 (pid 25598)] Task finished successfully.
2023-05-24 02:01:44.931 [38/validate/162 (pid 25643)] Task is starting.
2023-05-24 02:03:06.790 [38/validate/162 (pid 25643)] Task finished successfully.
2023-05-24 02:03:07.722 [38/end/163 (pid 25705)] Task is starting.
2023-05-24 02:03:08.924 [38/end/163 (pid 25705)] challen

In [40]:
! python ../flows/cloud/taxi_challenger.py --environment=conda --production --branch new_challenger argo-workflows create


Metaflow 2.9.2+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxifare_prediction, Branch: prod.new_challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taxifareprediction.prod.newchallenger.taxifareprediction to Argo Workflows...

The namespace of this production flow is
    production:mfprj-yvz456dx2l3tfnse-0-sviu
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-yvz456dx2l3tfnse-0-sviu")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-yvz456dx2l3tfnse-0-sviu
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping conda environment...(this could take a few minutes)
Workflow taxifareprediction.prod.newchallenger.taxifareprediction 

In [36]:
! python ../flows/cloud/taxi_challenger.py --environment=conda --production --branch new_challenger argo-workflows trigger

Metaflow 2.9.2+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxifare_prediction, Branch: prod.newchallenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taxifareprediction.prod.newchallenger.taxifareprediction triggered on Argo Workflows (run-id argo-taxifareprediction.prod.newchallenger.taxifareprediction-k9kg2).
See the run in the UI at https://ui-pw-555244755.outerbounds.dev/TaxiFarePrediction/argo-taxifareprediction.prod.newchallenger.taxifareprediction-k9kg2


In [41]:
from metaflow import Flow, namespace
import numpy as np

NA = {
    'champion': 'production:mfprj-bl5jeer7xehnsiac-0-hqum',
    'new_challenger': 'production:mfprj-a2atl4ukaehxrfie-0-uate'
}

for model, n in NS.items():
    namespace(n)
    run = Flow('TaxiFarePrediction').latest_successful_run
    score = np.mean(run.data.scores)
    print(f"{model} accuracy = {100*score:.2f}%")

champion accuracy = 89.51%
challenger accuracy = 90.53%


## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected? 
- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
    - What gives you confidence one model is better than another?
    - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?  

## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!